In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy import stats
from pathlib import Path

import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, closed_loop_rsof, get_session_list, depth_decoder
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 6
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"fig2").mkdir(parents=True, exist_ok=True)

In [ ]:
# LOAD DATA
# Load an example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH8.2h_S20230116"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
# Concatenate all sessions for closedloop rsof
models = ["gof", "gadd", "g2d", "gratio"]
model_labels = ["OF only", "Additive", "Conjunctive", "Pure depth"]
cols = [
    "roi",
    "best_depth",
    "preferred_depth_closedloop",
    "preferred_depth_closedloop_crossval",
    "depth_tuning_popt_closedloop",
    "depth_tuning_test_rsq_closedloop",
    "depth_tuning_test_spearmanr_pval_closedloop",
    "depth_tuning_test_spearmanr_rval_closedloop",
    "preferred_RS_closedloop_g2d",
    "preferred_RS_closedloop_crossval_g2d",
    "preferred_OF_closedloop_g2d",
    "preferred_OF_closedloop_crossval_g2d",
]
cols_to_add = [
    "rsof_test_rsq_closedloop_",
    "rsof_rsq_closedloop_",
    "rsof_popt_closedloop_",
]
for model in models:
    for col in cols_to_add:
        cols.append(f"{col}{model}")

mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"])]
# results_all = v1_common_utils.concatenate_all_neurons_df(
#     flexilims_session=flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session,
#         exclude_openloop=False,
#         exclude_pure_closedloop=False,
#         v1_only=True,
#         trialnum_min=10,
#         mouse_list=mouse_list,
#     ),
#     filename="neurons_df.pickle",
#     cols=cols,
#     read_iscell=True,
#     verbose=True,
# )
# results_all["preferred_RS_closedloop_crossval_g2d"] = (
#     results_all["preferred_RS_closedloop_crossval_g2d"] * 100
# )
# results_all["preferred_depth_closedloop_crossval"] = (
#     results_all["preferred_depth_closedloop_crossval"] * 100
# )
# results_all["preferred_OF_closedloop_crossval_g2d"] = np.degrees(
#     results_all["preferred_OF_closedloop_crossval_g2d"]
# )
# results_all.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_closedloop.pickle")
results_all = pd.read_pickle(SAVE_ROOT / "fig2" /  "results_all_rsof_closedloop.pickle")

In [ ]:
# Load example session
project = "hey2_3d-vision_foodres_20220101"
session_name_openloop = "PZAH6.4b_S20220524"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example_openloop, trials_df_example_openloop = spheres.sync_all_recordings(
    session_name=session_name_openloop,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

neurons_ds_example_openloop = pipeline_utils.create_neurons_ds(
    session_name=session_name_openloop,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example_openloop = pd.read_pickle(neurons_ds_example_openloop.path_full)

# Concatenate all sessions for openloop
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"
                ])]
# results_all_openloop = v1_common_utils.concatenate_all_neurons_df(
#     flexilims_session=flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session,
#         exclude_openloop=False,
#         exclude_pure_closedloop=True,
#         mouse_list=mouse_list,
#         v1_only=True,
#         trialnum_min=10,
#     ),
#     filename="neurons_df.pickle",
#     cols=[
#         "roi",
#         "best_depth",
#         "preferred_depth_closedloop",
#         "preferred_depth_closedloop_crossval",
#         "depth_tuning_test_rsq_closedloop",
#         "depth_tuning_test_spearmanr_pval_closedloop",
#         "depth_tuning_test_spearmanr_rval_closedloop",
#         "preferred_RS_closedloop_g2d",
#         "preferred_RS_closedloop_crossval_g2d",
#         "preferred_OF_closedloop_g2d",
#         "preferred_OF_closedloop_crossval_g2d",
#         "rsof_rsq_closedloop_crossval_g2d",
#         "rsof_rsq_closedloop_g2d",
#         "preferred_RS_openloop_actual_g2d",
#         "preferred_OF_openloop_actual_g2d",
#         "rsof_rsq_openloop_actual_g2d",
#         "rsof_popt_closedloop_g2d",
#         "rsof_popt_openloop_actual_g2d",
#     ],
#     read_iscell=True,
#     verbose=True,
# )
# results_all_openloop["preferred_RS_closedloop_g2d"] = (
#     results_all_openloop["preferred_RS_closedloop_g2d"] * 100
# )
# results_all_openloop["preferred_depth_closedloop"] = (
#     results_all_openloop["preferred_depth_closedloop"] * 100
# )
# results_all_openloop["preferred_OF_closedloop_g2d"] = np.degrees(
#     results_all_openloop["preferred_OF_closedloop_g2d"]
# )
# results_all_openloop["preferred_RS_openloop_actual_g2d"] = (
#     results_all_openloop["preferred_RS_openloop_actual_g2d"] * 100
# )
# results_all_openloop["preferred_OF_openloop_actual_g2d"] = np.degrees(
#     results_all_openloop["preferred_OF_openloop_actual_g2d"]
# )

# results_all_openloop.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_openloop.pickle")
results_all_openloop = pd.read_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_openloop.pickle")
print(f"openloop sessions {len(results_all_openloop.session.unique())}")
print(f"5 depth sessions {np.sum(pd.Series(results_all_openloop.session.unique()).str.contains('PZAH6.4b|PZAG3.4f'))}")

In [ ]:
# Load decoder results
# project = "hey2_3d-vision_foodres_20220101"
# flexilims_session = flz.get_flexilims_session(project)
# decoder_results = depth_decoder.concatenate_all_decoder_results(
#     flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session,
#         exclude_openloop=False,
#         exclude_pure_closedloop=True,
#         v1_only=True,
#         trialnum_min=10,
#     ),
#     filename="decoder_results.pickle",
# )
# decoder_results.to_pickle(SAVE_ROOT / "fig2" / "decoder_results_openloop.pickle")
decoder_results = pd.read_pickle(SAVE_ROOT / "decoder_results_openloop.pickle")

In [ ]:
# Fig. 3A-C: 1 example neuron, binned by RS (A), OF (B), and heatmap (C)
# Fig. 3D-F: another example neuron, binned by RS (D), OF (E), and heatmap (F)
EXAMPLE_ROIS = [
    # 402,
    #    86, 698,
    88,
]
fontsize_dict = {"title": 8, "label": 7, "tick": 5, "legend": 5}
model_labels = ["Optic flow", "Additive", "Conjunctive", "Pure depth"]
results_all["preferred_depth_amplitude"] = results_all[
    "depth_tuning_popt_closedloop"
].apply(lambda x: np.exp(x[0]) + x[-1])

# Filter depth neurons
neurons_df_sig = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.001)
    & (results_all["preferred_depth_amplitude"] > 0.5)
]
fig = plt.figure(figsize=(18 / 2.54, 21 / 2.54))
for iroi, roi in enumerate(EXAMPLE_ROIS):
    fig.add_axes([0.06, 0.88 - 0.43 * iroi, 0.12, 0.1])
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=2,
        linecolor="royalblue",
        markersize=5,
        closed_loop=1,
        fontsize_dict=fontsize_dict,
    )

    closed_loop_rsof.plot_speed_tuning(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="RS",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd=1,
        markersize=2,
        linewidth=1,
        plot_x=0.25,
        plot_y=0.88 - 0.43 * iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        legend_on=False,
    )
    ylim = plt.gca().get_ylim()

    closed_loop_rsof.plot_speed_tuning(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="OF",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd=1,
        markersize=2,
        linewidth=1,
        plot_x=0.44,
        plot_y=0.88 - 0.43 * iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        legend_on=True,
    )
    plt.gca().set_ylabel("")
    plt.gca().set_ylim(ylim)
    plt.gca().set_yticklabels([])

    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=None,
        vmax=None,
        xlabel="Running speed (cm/s)",
        ylabel="Optic flow speed (degrees/s)",
        plot_x=0.04,
        plot_y=0.67 - 0.43 * iroi,
        plot_width=0.13,
        plot_height=0.13,
        cbar_width=0.01,
        fontsize_dict=fontsize_dict,
    )

    for imodel, (model, model_label) in enumerate(zip(models, model_labels)):
        if imodel == 0:
            ylabel = "Optic flow speed (degrees/s)"
        else:
            ylabel = ""
        if imodel == 1:
            xlabel = "Running speed (cm/s)"
        else:
            xlabel = ""

        closed_loop_rsof.plot_RS_OF_fit(
            fig=fig,
            neurons_df=neurons_df_example,
            roi=roi,
            model=model,
            model_label=model_label,
            min_sigma=0.25,
            vmin=vmin,
            vmax=vmax,
            log_range={
                "rs_bin_log_min": 0,
                "rs_bin_log_max": 2.5,
                "rs_bin_num": 6,
                "of_bin_log_min": -1.5,
                "of_bin_log_max": 3.5,
                "of_bin_num": 11,
                "log_base": 10,
            },
            plot_x=0.24 + 0.1 * imodel,
            plot_y=0.67 - 0.43 * iroi,
            plot_width=0.13,
            plot_height=0.13,
            xlabel=xlabel,
            ylabel=ylabel,
            fontsize_dict=fontsize_dict,
            cbar_width=None,
        )
        if imodel > 0:
            plt.gca().set_yticklabels([])

# Fig. : compare R2 of different models
closed_loop_rsof.plot_r2_comparison(
    fig=fig,
    neurons_df=neurons_df_sig,
    models=models,
    labels=model_labels,
    plot_type="bar",
    markersize=3,
    alpha=0.7,
    color="k",
    plot_x=0.75, #0.75
    plot_y=0.76, #0.72
    plot_width=0.2,
    plot_height=0.2,
    fontsize_dict=fontsize_dict,
)
plt.ylim([-0.02,1.02])

# # fig.add_axes([0.75, 0.27, 0.2, 0.2])
# # closed_loop_rsof.plot_r2_violin(
# #     neurons_df_sig,
# #     models=models,
# #     model_labels=model_labels,
# #     fontsize_dict=fontsize_dict,
# #     ylim=[10**-3, 1],
# # )
# # closed_loop_rsof.plot_scatter(
# #     fig,
# #     neurons_df_sig,
# #     "rsof_rsq_closedloop_gadd",
# #     "rsof_rsq_closedloop_g2d",
# #     xlabel="Running speed (cm/s)",
# #     ylabel="Preferred depth (cm)",
# #     s=10,
# #     alpha=0.2,
# #     c="k",
# #     plot_x=0.75,
# #     plot_y=0.27,
# #     plot_width=0.2,
# #     plot_height=0.2,
# #     aspect_equal=True,
# #     plot_diagonal=True,
# #     diagonal_color="r",
# #     fontsize_dict=fontsize_dict,
# #     log_scale=False,
# # )

# Plot ratio - preferred depth summary plot
results_all["RS_OF_ratio_closedloop_crossval_g2d"] = results_all[
    "preferred_RS_closedloop_crossval_g2d"
] / np.radians(results_all["preferred_OF_closedloop_crossval_g2d"])


neurons_df = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0)
    & (results_all["rsof_rsq_closedloop_g2d"] > 0.02)
]

scatter_props = {
    "s": 1,
    "alpha": 0.2,
    "plot_diagonal": False,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}

r, p = closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df,
    xcol="RS_OF_ratio_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Ratio of preferred running \nand optic flow speeds (cm)",
    ylabel="Preferred virtual \ndepth (cm)",
    plot_x=0.07,
    plot_y=0.48,
    plot_width=0.21,
    plot_height=0.2,
    # aspect_equal=True,
    # plot_diagonal=False,
    # fontsize_dict=fontsize_dict,
    color="k",
    linewidth=1,
    # alpha=0.1,
    **scatter_props,
)

# Scatterplot of preferred RS and OF speeds (cross-validated) colored by preferred depth
closed_loop_rsof.plot_speed_colored_by_depth(
    fig=fig,
    neurons_df=neurons_df,
    xcol="preferred_RS_closedloop_crossval_g2d",
    ycol="preferred_OF_closedloop_crossval_g2d",
    zcol="preferred_depth_closedloop_crossval",
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred optic flow speed (degrees/s)",
    zlabel="Preferred virtual \ndepth (cm)",
    s=3,
    alpha=1,
    cmap="cool_r",
    plot_x=0.07,
    plot_y=0.2,
    plot_width=0.16,
    plot_height=0.35,
    fontsize_dict=fontsize_dict,
    edgecolors="white",
)

# Plot open-closedloop RS / OF trace
positions=[]
x0, y0, width, height = 0.32, 0.48, 0.12, 0.03
for i in range(4):
    positions.append([x0 + i//2 * width*1.1, y0 - i%2*height*1.3, width, height])
closed_loop_rsof.plot_speed_trace_closed_open_loop(flexilims_session=flexilims_session, 
                                  session_name=session_name_openloop, 
                                  trials_df=trials_df_example_openloop, 
                                  trial_list = [25,27,32], 
                                  positions=positions,
                                  linewidth=0.5,
                                  fontsize_dict={"title": 7, "label": 5, "tick": 5, "legend": 5})

# Plot example cell RS OF matrix (closed vs open loop)
# Plot example ROI depth tuning and RS-OF matrix
EXAMPLE_ROIS_OPENLOOP = [387]
for iroi, roi in enumerate(EXAMPLE_ROIS_OPENLOOP):
    if iroi != (len(EXAMPLE_ROIS_OPENLOOP) - 1):
        xlabel = ""
    else:
        xlabel = "Running speed (cm/s)"
    if iroi != (len(EXAMPLE_ROIS_OPENLOOP) // 2):
        ylabel = ""
    else:
        ylabel = "Optic flow speed (degrees/s)"
#     fig.add_axes([0.71, 0.59, 0.03, 0.05])
#     depth_selectivity.plot_depth_tuning_curve(
#         neurons_df=neurons_df_example_openloop,
#         trials_df=trials_df_example_openloop,
#         use_col="depth_tuning_popt_closedloop",
#         roi=roi,
#         rs_thr=None,
#         plot_fit=True,
#         plot_smooth=False,
#         linewidth=1,
#         linecolor="royalblue",
#         markersize=3,
#         closed_loop=1,
#         fontsize_dict=fontsize_dict,
#         # ylim=(-0.01,0.5)
# )   
#     # if iroi != len(EXAMPLE_ROIS_OPENLOOP) - 1:
#     plt.xlabel("")

    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example_openloop,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=None,
        vmax=None,
        xlabel="",
        ylabel=ylabel,
        plot_x=0.66,
        plot_y=0.47 - 0.3 * iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=None,
        fontsize_dict=fontsize_dict,
    )

    closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example_openloop,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=0,
        vmin=vmin,
        vmax=vmax,
        xlabel=xlabel,
        ylabel="",
        plot_x=0.83,
        plot_y=0.47 - 0.3 * iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=0.005,
        fontsize_dict=fontsize_dict,
    )

# Plot preferred RS & OF closedloop vs openloop
results_all_openloop["amplitude_closedloop"] = results_all_openloop["rsof_popt_closedloop_g2d"].apply(
    lambda x: np.exp(x[0]) + x[-1]
)
results_all_openloop["amplitude_openloop"] = results_all_openloop["rsof_popt_openloop_actual_g2d"].apply(
    lambda x: np.exp(x[0]) + x[-1]
)
neurons_df_sig = results_all_openloop[
    (results_all_openloop["iscell"] == 1)
    & (results_all_openloop["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all_openloop["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all_openloop["rsof_test_rsq_closedloop_g2d"] > 0.02)
]

scatter_props = {
    "s": 3,
    "alpha": 0.2,
    "plot_diagonal": True,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}
closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df_sig,
    xcol="preferred_RS_closedloop_g2d",
    ycol="preferred_RS_openloop_actual_g2d",
    xlabel="Preferred running speed \nin closed loop(cm/s)",
    ylabel="Preferred running speed \nin open loop (cm/s)",
    color="g",
    plot_x=0.37,
    plot_y=0.28,
    plot_width=0.12,
    plot_height=0.12,
    **scatter_props,
)

closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df_sig,
    xcol="preferred_OF_closedloop_g2d",
    ycol="preferred_OF_openloop_actual_g2d",
    xlabel="Preferred optic flow speed \nin closed loop (degrees/s)",
    ylabel="Preferred optic flow speed \nin open loop (degrees/s)",
    color="b",
    plot_x=0.6,
    plot_y=0.28,
    plot_width=0.12,
    plot_height=0.12,
    **scatter_props,
)

# Plot scatter of closedloop / openloop amplitude
# closed_loop_rsof.plot_2d_hist(
#     fig=fig,
#     neurons_df=neurons_df_sig,
#     xcol="amplitude_closedloop",
#     ycol="amplitude_openloop",
#     xlabel="Amplitude in closed loop",
#     ylabel="Amplitude in open loop",
#     color="r",
#     plot_x=0.82,
#     plot_y=0.28,
#     plot_width=0.12,
#     plot_height=0.12,
#     **scatter_props,
# )
scatter_props = {
    "s": 2,
    "alpha": 0.05,
    "plot_diagonal": True,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}
closed_loop_rsof.plot_scatter(
    fig=fig,
    neurons_df=neurons_df_sig,
    xcol="amplitude_closedloop",
    ycol="amplitude_openloop",
    xlabel="Amplitude in closed loop",
    ylabel="Amplitude in open loop",
    c="r",
    plot_x=0.82,
    plot_y=0.28,
    plot_width=0.12,
    plot_height=0.12,
    **scatter_props,
    diagonal_linewidth=2,
)

# Plot decoding results 

# Plot accuracy
fig.add_axes([0.05, 0.03, 0.25, 0.18])
# Plot accuracy
decoder_results=decoder_results[decoder_results.acc_speed_bins_closedloop.notnull()]
depth_decoder.decoder_accuracy(
    decoder_results,
    markersize=3,
    colors=["k", "k"],
    markers=["o", "^"],
    markerfacecolors=["k", "none"],
    linewidth=1,
    xlabel=["Closed loop", "Open loop"],
    ylabel="Decoding accuracy",
    fontsize_dict=fontsize_dict,
) 

# Plot conmat
ndepths = 8
conmat_mean = depth_decoder.calculate_average_confusion_matrix(
    decoder_results[decoder_results["ndepths"] == ndepths]
)

ax1=fig.add_axes([0.45, 0.03, 0.18, 0.18])
ax2=fig.add_axes([0.7, 0.03, 0.18, 0.18])
depth_decoder.plot_closed_open_conmat(
    conmat_mean,
    normalize=True,
    fig=fig,
    ax1=ax1,
    ax2=ax2,
    fontsize_dict=fontsize_dict,
)
plt.savefig(SAVE_ROOT / "fig2.pdf", dpi=300)
plt.savefig(SAVE_ROOT / "fig2.png", dpi=300)

In [ ]:
fontsize_dict = {"title": 8, "label": 7, "tick": 5, "legend": 5}
model_labels = ["Optic flow", "Additive", "Conjunctive", "Pure depth"]
results_all["preferred_depth_amplitude"] = results_all[
    "depth_tuning_popt_closedloop"
].apply(lambda x: np.exp(x[0]) + x[-1])

# Filter depth neurons
neurons_df_sig = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.001)
    # & (results_all["preferred_depth_amplitude"] > 0.5)
]

# Fig. : compare R2 of different models
fig = plt.figure()
closed_loop_rsof.plot_r2_comparison(
    fig=fig,
    neurons_df=neurons_df_sig,
    models=models,
    labels=model_labels,
    plot_type="bar",
    markersize=3,
    alpha=0.5,
    color="k",
    plot_x=0.75, #0.75
    plot_y=0.76, #0.72
    plot_width=0.2,
    plot_height=0.2,
    fontsize_dict=fontsize_dict,
)
plt.ylim([-0.02,1.02])

In [ ]:
results_all_openloop["amplitude_closedloop"] = results_all_openloop["rsof_popt_closedloop_g2d"].apply(
    lambda x: np.exp(x[0]) + x[-1]
)
results_all_openloop["amplitude_openloop"] = results_all_openloop["rsof_popt_openloop_actual_g2d"].apply(
    lambda x: np.exp(x[0]) + x[-1]
)
neurons_df_sig = results_all_openloop[
    (results_all_openloop["iscell"] == 1)
    & (results_all_openloop["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all_openloop["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all_openloop["rsof_rsq_closedloop_g2d"] > 0.02)
]
scatter_props = {
    "s": 2,
    "alpha": 0.1,
    "plot_diagonal": True,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}
fig = plt.figure()
closed_loop_rsof.plot_scatter(
    fig=fig,
    neurons_df=neurons_df_sig,
    xcol="amplitude_closedloop",
    ycol="amplitude_openloop",
    xlabel="Amplitude in closed loop",
    ylabel="Amplitude in open loop",
    c="r",
    plot_x=0.82,
    plot_y=0.28,
    plot_width=0.4,
    plot_height=0.4,
    **scatter_props,
    diagonal_linewidth=2,
)

In [ ]:
neurons_df_sig = results_all_openloop[
    (results_all_openloop["iscell"] == 1)
    & (results_all_openloop["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all_openloop["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    # & ((results_all_openloop["rsof_rsq_closedloop_g2d"] > 0.02)|(results_all_openloop["rsof_rsq_openloop_actual_g2d"] > 0.02))
]
ratios = neurons_df_sig["amplitude_closedloop"] / neurons_df_sig["amplitude_openloop"]
bins = np.geomspace(1e-2, 1e2, 31)
fig = plt.figure()
plt.hist(ratios, bins=bins, color="k", alpha=0.5)
plt.xscale("log")
plt.vlines(1, 0, 4000, color="r")
print(np.nanmedian(ratios))

In [ ]:
decoder_results

In [ ]:
# compare RS OF fit stats
print(stats.wilcoxon(neurons_df_sig["amplitude_closedloop"], neurons_df_sig["amplitude_openloop"]))
print(stats.wilcoxon(neurons_df_sig["amplitude_closedloop"], neurons_df_sig["amplitude_openloop"], alternative="greater"))

print(stats.wilcoxon(neurons_df_sig["preferred_RS_closedloop_g2d"], neurons_df_sig["preferred_RS_openloop_actual_g2d"]))
print(stats.wilcoxon(neurons_df_sig["preferred_RS_closedloop_g2d"], neurons_df_sig["preferred_RS_openloop_actual_g2d"], alternative="greater"))

print(stats.wilcoxon(neurons_df_sig["preferred_OF_closedloop_g2d"], neurons_df_sig["preferred_OF_openloop_actual_g2d"]))
print(stats.wilcoxon(neurons_df_sig["preferred_OF_closedloop_g2d"], neurons_df_sig["preferred_OF_openloop_actual_g2d"], alternative="greater"))
print(stats.wilcoxon(neurons_df_sig["preferred_OF_closedloop_g2d"], neurons_df_sig["preferred_OF_openloop_actual_g2d"], alternative="less"))

In [ ]:
# stats comparison between best models
results_all["preferred_depth_amplitude"] = results_all[
    "depth_tuning_popt_closedloop"
].apply(lambda x: np.exp(x[0]))

# Filter depth neurons
neurons_df_sig = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.001)
    & (results_all["preferred_depth_amplitude"] > 0.5)
]

model_cols = [f"rsof_test_rsq_closedloop_{model}" for model in models]
# Find the best model for each neuron
neurons_df_sig["best_model"] = neurons_df_sig[model_cols].idxmax(axis=1)

# Calculate percentage of neurons that have the best model
neuron_sum = (
    neurons_df_sig.groupby("session")[["roi"]].agg(["count"]).values.flatten()
)
props = []
# calculate the proportion of neurons that have the best model for each session
for i, model in enumerate(model_cols):
    prop = (
        neurons_df_sig.groupby("session")
        .apply(lambda x: x[x["best_model"] == model][["roi"]].agg(["count"]))
        .values.flatten()
    ) / neuron_sum
    props.append(prop)
# with open(SAVE_ROOT / "fig2" / "rsof_bestmodel_prop.pkl", "wb") as f:
#     pickle.dump(props, f)
# with open(SAVE_ROOT / "fig2" / "rsof_bestmodel_prop.pkl", "rb") as f:
#     props = pickle.load(f)
for i in [0,1,3]:
    print(stats.wilcoxon(props[i], props[2], alternative="less"))

In [ ]:
# CHECK:
# - what's going on with the purple cells at the bottom right corner of the scatter plot?

# SUPP